This snippets are taken from book Python Geospatial Development - Third Edition

In [ ]:
# Calculate distance using pyproj
import pyproj

lat1,long1 = (37.8101274,-122.4104622)
lat2,long2 = (37.80237485,-122.405832766082)

geod = pyproj.Geod(ellps="WGS84")
angle1,angle2,distance = geod.inv(long1, lat1, long2, lat2)

print(f"Distance is {distance: .2f} meters")

In [ ]:
# Using OSGEO , GDAL to look at the contents of the shapefile
# USA census data can be found here to download - https://www2.census.gov/geo/tiger/TIGER2021/STATE/

shp_path = "data/tl_2021_us_state/tl_2021_us_state.shp"
import osgeo.ogr

shapefile = osgeo.ogr.Open("data/tl_2021_us_state/tl_2021_us_state.shp")
numLayers = shapefile.GetLayerCount()

print("Shapefile contains {} layers".format(numLayers))
print()

for layerNum in range(numLayers):
  layer = shapefile.GetLayer(layerNum)
  spatialRef = layer.GetSpatialRef().ExportToProj4()
  numFeatures = layer.GetFeatureCount()
  print("Layer {} has spatial reference {}".format(
        layerNum, spatialRef))
  print("Layer {} has {} features:".format(
        layerNum, numFeatures))
  print()

  for featureNum in range(numFeatures):
    feature = layer.GetFeature(featureNum)
    featureName = feature.GetField("NAME")

    print("Feature {} has name {}".format(featureNum, featureName))

In [ ]:
"""
A geometry object is a complex structure that holds some geospatial data, 
often using nested geometry objects to reflect the way the geospatial data is organized. 
So far, we've discovered that New Mexico's geometry consists of a polygon.
"""
import osgeo.ogr

def analyzeGeometry(geometry, indent=0):
  s = []
  s.append("  " * indent)
  s.append(geometry.GetGeometryName())
  if geometry.GetPointCount() > 0:
    s.append(" with {} data points".format(geometry.GetPointCount()))
  if geometry.GetGeometryCount() > 0:
    s.append(" containing:")

  print("".join(s))

  for i in range(geometry.GetGeometryCount()):
    analyzeGeometry(geometry.GetGeometryRef(i), indent+1)

shapefile = osgeo.ogr.Open("data/tl_2021_us_state/tl_2021_us_state.shp")
layer = shapefile.GetLayer(0)
feature = layer.GetFeature(12)
geometry = feature.GetGeometryRef()

analyzeGeometry(geometry)

In [ ]:
"""
what is the distance from the northernmost point to the southernmost point in California? 
There are various ways we could answer this question, but for now, we'll do it by hand.
"""

import osgeo.ogr

shp_path = "data/tl_2021_us_state/tl_2021_us_state.shp"

def findPoints(geometry, results):
  for i in range(geometry.GetPointCount()):
    x,y,z = geometry.GetPoint(i)
    if results['north'] == None or results['north'][1] < y:
      results['north'] = (x,y)
    if results['south'] == None or results['south'][1] > y:
      results['south'] = (x,y)

  for i in range(geometry.GetGeometryCount()):
    findPoints(geometry.GetGeometryRef(i), results)

shapefile = osgeo.ogr.Open(shp_path)
layer = shapefile.GetLayer(0)
feature = layer.GetFeature(13)
geometry = feature.GetGeometryRef()

results = {'north' : None,
           'south' : None}

findPoints(geometry, results)

print("Northernmost point is ({:.4f}, {:.4f})".format(
      results['north'][0], results['north'][1]))
print("Southernmost point is ({:.4f}, {:.4f})".format(
      results['south'][0], results['south'][1]))

In [ ]:
"""
how you can use GDAL to write raster-format data into a GeoTIFF formatted file. 
For our example, we'll divide the entire Earth's surface into 360 cells horizontally and 180 cells vertically, 
so that each cell covers one degree of latitude and longitude. 
For each cell, we'll store a single random number between 1 and 100.
"""
# Create Raster file
from osgeo import gdal
driver = gdal.GetDriverByName("GTIFF")
dstFile = driver.Create("Example Raster.tiff", 360, 180, 1, gdal.GDT_Int16)

# 
from osgeo.gdal import osr
spatialReference = osr.SpatialReference()
spatialReference.SetWellKnownGeogCS("WGS84")
dstFile.SetProjection(spatialReference.ExportToWkt())

originX    = -180
originY    = 90
cellWidth  = 1.0
cellHeight = 1.0

dstFile.SetGeoTransform([originX, cellWidth, 0,
                         originY, 0, -cellHeight])

band = dstFile.GetRasterBand(1)

import random

values = []
for row in range(180):
    row_data = []
    for col in range(360):
        row_data.append(random.randint(1, 100))
    values.append(row_data)

import struct

fmt = "<" + ("h" * band.XSize)

for row in range(180):
    scanline = struct.pack(fmt, *values[row])
    band.WriteRaster(0, row, 360, 1, scanline)

# the tiff file now contains gdal raster data
# easier to write using numpy
import numpy

array = numpy.array(values, dtype=numpy.int16)
band.WriteArray(array)

In [ ]:
import pyproj
print(pyproj.__version__)

In [ ]:
# Shapely example
import shapely.geometry
import shapely.wkt

pt = shapely.geometry.Point(0, 0)
circle = pt.buffer(1.0)

square = shapely.geometry.Polygon([(0, 0), (1, 0),
                                   (1, 1), (0, 1),
                                   (0, 0)])

intersect = circle.intersection(square)
print(shapely.wkt.dumps(intersect))

In [ ]:
"""
Connect PG
Load Shape file into POSTGIS

"""

import psycopg2
from osgeo import ogr

shp_file_path = "data/TM_WORLD_BORDERS-0.3/TM_WORLD_BORDERS-0.3.shp"

connection = psycopg2.connect(
    host="localhost",
    port=5432,
    database="test",
    user="postgres",
    password="fr24Password")

cursor = connection.cursor()

cursor.execute("DROP TABLE IF EXISTS borders")
cursor.execute("CREATE TABLE borders (" +
               "id SERIAL PRIMARY KEY," +
               "name VARCHAR NOT NULL," +
               "iso_code VARCHAR NOT NULL," +
               "outline GEOGRAPHY)")
cursor.execute("CREATE INDEX border_index ON borders " +
               "USING GIST(outline)")
connection.commit()

In [ ]:
shapefile = ogr.Open(shp_file_path)
layer = shapefile.GetLayer(0)

for i in range(layer.GetFeatureCount()):
    feature  = layer.GetFeature(i)
    name     = feature.GetField("NAME")
    iso_code = feature.GetField("ISO3")
    geometry = feature.GetGeometryRef()
    wkt      = geometry.ExportToWkt()

    cursor.execute("INSERT INTO borders (name, iso_code, outline) " +
                   "VALUES (%s, %s, ST_GeogFromText(%s))",
                   (name, iso_code, wkt))

connection.commit()

In [ ]:
start_long = 8.542
start_lat  = 47.377
radius     = 500000

cursor.execute("SELECT name FROM borders WHERE ST_DWithin(" +
               "ST_MakePoint(%s, %s), outline, %s)",
               (start_long, start_lat, radius))
for row in cursor:
    print(row[0])